In [1]:
# !pip install openpyxl
# !pip install python-docx

In [2]:
import pandas as pd
import numpy as np
import os
import docx
import sys
import traceback
import datetime

from openpyxl import load_workbook, Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils.cell import coordinate_from_string

import time
import logging

## Coded string reader function

In [3]:
def read(doc_obj):
    '''
    This function reads the coded string in a docx file to a dataframe
    '''
    
    try:
        # with open(read_path, 'r') as file:
        #     coded_string = file.read()
        if len(doc_obj.paragraphs) > 1:
            print(f"At this action {doc_obj.paragraphs[1].text.split(',')[0]}, there is an issue. Please Check!")
            sys.exit(1)
        else:                  
            for paragraph in doc_obj.paragraphs:
                coded_string = paragraph.text
                coded_string_list = coded_string.split(',')
                df = pd.DataFrame(coded_string_list, columns = ['strings'])
                df[['team', 'jersey_number', 'action', 'notation', 'start_grid', 'end_grid', 'timestamp', 'foot', 'special_attribute', 'half']] = df['strings'].str.split('-', expand = True)
                df = df.drop(columns = ['strings'])
                return df
                
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()


In [4]:
# read_path = r'..\read_string\rstring.txt'
doc_obj = docx.Document(r"..\read_string\rstring.docx")
df = read(doc_obj)

## Renumeration log function

In [5]:
match_idd = int(input('Match ID please... = '))

def renumeration_func():
    '''
    This function creates a renumeration excel file as per the inputs given.
    It checks the match id to make sure no matches are duplicated.
    '''
    try:

        analyst_names = {
            1 : 'Sreyas',
            2 : 'Boni',
            3 : 'Arpit',
            4 : 'Sudhanva'
        }
        
        write_path_xlsx = r'..\write_string\renumeration.xlsx'
        initial_input = input('Do you wish to add match details? (y/n)')

        # Taking the match details from the user
        if initial_input.lower() == 'y':
            global team_a_name
            global team_b_name
            analyst_id = int(input(f'Choose the Analyst ID\n{analyst_names} = '))
            analyst_name = analyst_names[analyst_id]
            team_a_name = input('Team A name = ')
            team_b_name = input('Team B name = ')
            match_id = match_idd
            game_time = int(input('Game time?\nType 1 for 90 minutes\nType 2 for 45 minutes\nType 3 for less than 60 minutes\n'))
            current_date = datetime.date.today()
            
            #check to-do 
            if game_time == 1:
                renumeration = 500
                game_time = 90
            elif game_time == 2:
                renumeration = 250
                game_time = 45
            elif game_time == 3:
                renumeration = 300
                game_time = 'Less than 60'
            else:
                print('You have chosen an invalid option. Try again!')
        
            data = {
                    'team_a_name' : team_a_name,
                    'team_b_name' : team_b_name,
                    'match_id' : match_id,
                    'game_time' : game_time,
                    'current_date' : current_date,
                    'renumeration' : renumeration
                }        
            
            # There are 4 possible scenarios here
            # 1. The file exists
            if os.path.exists(write_path_xlsx):
                wb = load_workbook(write_path_xlsx)
                sheet_name = analyst_name
                sheet_exists = sheet_name in wb.sheetnames
                
                # 2. File exists but sheet doesn't  
                if not sheet_exists:                
                    ws = wb.create_sheet(sheet_name)                
                    renumeration_df = pd.DataFrame(data, index = [0])
                    for r in dataframe_to_rows(renumeration_df, index=False, header=True):
                        ws.append(r)
                    for cell in ws[1]:
                        cell.style = 'Pandas'
                    wb.save(write_path_xlsx)
                    
                # 3. File and sheet exists
                else:                  
                    renumeration_df = pd.read_excel(write_path_xlsx, sheet_name = analyst_name)
                    renumeration_df.loc[len(renumeration_df)] = data
                    if renumeration_df['match_id'].duplicated().sum() > 0:
                        display(renumeration_df[renumeration_df['match_id'].duplicated()])
                        print('This match id seems to be a duplicate. Please check!')
                        # renumeration_df = renumeration_df.drop_duplicates(subset = ['match_id'])
                    else:
                        renumeration_df = pd.DataFrame(data, index = [0])
                        for r in dataframe_to_rows(renumeration_df, index=False, header=False):
                            ws = wb[sheet_name]
                            ws.append(r)
                        wb.save(write_path_xlsx)
                        
            # 4. File does not exists 
            else:
                renumeration_df = pd.DataFrame(data, index = [0])
                renumeration_df.to_excel(write_path_xlsx, sheet_name = analyst_name, index = False)     
                
           
            
                
        else:
            print('Proceeding without filling details')

    except PermissionError as e:
        print(f'{e}\nThe renumeration xlsx file might be open. Please close it and try again')

    except Exception as e:
        print(f'An error occured : {e}')

Match ID please... =  777555


In [6]:
renumeration_func()

Do you wish to add match details? (y/n) y
Choose the Analyst ID
{1: 'Sreyas', 2: 'Boni', 3: 'Arpit', 4: 'Sudhanva'} =  1
Team A name =  ghana
Team B name =  cuba
Game time?
Type 1 for 90 minutes
Type 2 for 45 minutes
Type 3 for less than 60 minutes
 1


## Excel error logs

In [7]:

path = r'..\excel_logs'
if not os.path.exists(path):
    os.makedirs(path)
    
qc_path_xlsx = "../excel_logs/Match_ID_" + str(match_idd) + "_Time_" + time.strftime("%H-%M-%S") + ".xlsx"

def qc_excel_log(df, message, size, sheet_name):
    
    if os.path.exists(qc_path_xlsx):
        wb = load_workbook(qc_path_xlsx)
        ws = wb.create_sheet(f'{sheet_name}')

    else:
        wb = Workbook()
        ws = wb.active
        ws.title = f'{sheet_name}'


    a1 = ws['A1']
    a1.value = f'{message}'
    
    a1.font = Font(size=12, underline='single')
    a1.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    
    for row in dataframe_to_rows(df, index=False, header=True):
        ws.append(row)
    
    for cell in ws[2]:
        cell.style = 'Pandas'

        
    for row in ws.iter_rows():
        for cell in row:
            if cell.value:
                max_cell = cell
    
    coordinate = coordinate_from_string(max_cell.coordinate)
    max_col = coordinate[0]
    max_row = coordinate[1]
    
    ws.move_range(f"A2:{max_col}{max_row}", rows=0, cols=1)
    
    ws.column_dimensions['A'].width = size

    wb.save(qc_path_xlsx)

## Logger function

In [8]:
def logger_setup(filename):
    logging.basicConfig(filename=filename, filemode="w", level=logging.DEBUG)
    log_obj = logging.getLogger()
    return log_obj

In [9]:
logs_path = r'..\logs'
if not os.path.exists(logs_path):
    os.makedirs(logs_path)

logger_obj = logger_setup("../logs/QC_Match_ID_" + str(match_idd) + "_Time_" + time.strftime("%H-%M-%S") + ".log")

## Dataframe to coded string function

In [10]:
def write(write_path, df):
    '''
    This function creates a coded string from a dataframe
    '''
    
    if not 'Combined' in df.columns:
        df['Combined'] = df.apply(lambda row: '-'.join(row), axis=1)
        cs_list = list()
        df['Combined'].apply(lambda x : cs_list.append(x))
        
        if not os.path.exists(write_path):
            with open(write_path, 'a') as file:
                for item in cs_list:
                    if item == cs_list[-1]:
                        file.write(item)
                    else:
                        file.write(item + ',')

            print("wstring file successfully created.")
    
        else:
            print("This file already exists")

    else:
        print("Read the file again. Combined column already exists.")

In [11]:
write_path = "../write_string/wstring/Match_ID_" + str(match_idd) + "_Time_" + time.strftime("%H-%M-%S") + ".txt"
# write(write_path)

## Dataframe to coded string function mini

In [12]:
def write_log(log_df):
    '''
    This function creates a coded string from a dataframe for logger object
    '''
    log_df = log_df.copy()
    if not 'Combined' in log_df.columns:
        log_df['Combined'] = log_df.apply(lambda row: '-'.join(row), axis=1)
        cs_list = list()
        log_df['Combined'].apply(lambda x : cs_list.append(x))
        return cs_list

## Appropriate fouls QC function

In [13]:
def appropriate_foul():
    '''
    This function checks if 
    1. A non-defensive action(other than ST, SL, AD, GD, HB) was given a foul
    2. A successful defensive action was tagged as a foul(need to check for yc and rc)
    '''
    non_df_action = df[~(df['action'].isin(['ST', 'SL', 'AD', 'GD', 'HB'])) & (df['special_attribute'].isin(['F', 'YC', 'RC']))]
    non_df_action_foul = non_df_action['action'].count()

    successful_df_action = df[(df['action'].isin(['ST', 'SL', 'AD', 'GD']))  & (df['notation'] == '1') & (df['special_attribute'].isin(['F', 'YC', 'RC']))]
    successful_df_action_foul = successful_df_action['action'].count()
    
    if (non_df_action_foul != 0) | (successful_df_action_foul != 0):
        if (non_df_action_foul != 0):
            print('A non defensive action was tagged as a foul. Please check!')
            display(non_df_action)
            # logger_obj.info(f"\n\nA non defensive action was tagged as a foul. Please check! \n{write_log(non_df_action)}\n\n" + '*'*100)
            message = 'A non defensive action was tagged as a foul. Please check!'
            qc_excel_log(non_df_action, message, 30, 'non_df_action_foul')
        
        if (successful_df_action_foul != 0):
            print('A successful defensive action was tagged as a foul. Please check!')
            display(successful_df_action)
            # logger_obj.info(f"\n\nA successful defensive action was tagged as a foul. Please check! \n{write_log(successful_df_action)}\n\n" + '*'*100)
            message = 'A successful defensive action was tagged as a foul. Please check!'
            qc_excel_log(successful_df_action, message, 30, 'successful_df_action_foul')
    
    else:
        print('Appropriate foul QC done.')


In [14]:
appropriate_foul()

Appropriate foul QC done.


## Freekick-Penalty = Fouls QC function

In [15]:
def fk_pk_foul_check():
    '''
    This function finds the misbehaviour foul counts, subtracts them from total fouls and checks if total fouls equal
    total freekick-penalty
    '''
    # finding misbehaviour foul count
    misbehaviour_foul_a = 0
    misbehaviour_foul_b = 0
    
    st_0_list = df[(df['action'] == 'ST') & (df['notation'] == '0') & (df['special_attribute'].isin(['YC', 'RC']))].index
    for index in st_0_list:
        if ((index == df.index[-1]) or (df.loc[index + 1, 'action'] != 'XST')) & (df.loc[index, 'team'] == 'A'):
            misbehaviour_foul_a += 1
        elif ((index == df.index[-1]) or (df.loc[index + 1, 'action'] != 'XST')) & (df.loc[index, 'team'] == 'B'):
            misbehaviour_foul_b += 1

    # finding fk-pk and fouls count
    teamb_foul = df[(df['team'] == 'B') & ((df['action'].isin(['HB', 'OFF'])) | (df['special_attribute'].isin(['F', 'YC', 'RC'])))]['action'].count()
    teamb_foul = teamb_foul - misbehaviour_foul_b
    teamb_fk_pk = df[(df['team'] == 'B') & (df['special_attribute'].isin(['FK', 'PK']))]['action'].count()
    
    teama_foul = df[(df['team'] == 'A') & ((df['action'].isin(['HB', 'OFF'])) | (df['special_attribute'].isin(['F', 'YC', 'RC'])))]['action'].count()
    teama_foul = teama_foul - misbehaviour_foul_a
    teama_fk_pk = df[(df['team'] == 'A') & (df['special_attribute'].isin(['FK', 'PK']))]['action'].count()

    # verifying if the foul and fk-pk count match
    if (teamb_foul == teama_fk_pk) & (teama_foul == teamb_fk_pk):
        print("Foul to fk-pk QC done")
        logger_obj.info('\n\nFoul to fk-pk QC done\n\n' + '*'*100)
    else:
        print("Foul to fk-pk not equal")
        print(f"Team A FK-PK = {teama_fk_pk}, Team B Fouls = {teamb_foul}")
        print(f"Team B FK-PK = {teamb_fk_pk}, Team A Fouls = {teama_foul}")
        # logger_obj.info(f'''\n\n
        # Foul to fk-pk not equal
        # Team A FK-PK = {teama_fk_pk}, Team B Fouls = {teamb_foul}
        # Team B FK-PK = {teamb_fk_pk}, Team A Fouls = {teama_foul}\n
        # ''')
    
        teamB = (teamb_foul == teama_fk_pk)
        teamA = (teama_foul == teamb_fk_pk)
        if not (teamB | teamA):
# check to-do
            print('Both inequal. Check TMS')
        elif teamB:
            foul_team = 'A'
            fk_team = 'B'
        else:
            foul_team = 'B'
            fk_team = 'A'
# check to-do
        try:
            foul_mask = ( (df['team'] == foul_team) & ((df['action'].isin(['HB', 'OFF'])) | (df['special_attribute'].isin(['F', 'YC', 'RC']))) )
            fk_mask = ( (df['team'] == fk_team) & (df['special_attribute'].isin(['FK', 'PK'])) )
            display(df[foul_mask | fk_mask])
            message = f'''
            Foul to fk-pk not equal
            Team A FK-PK = {teama_fk_pk}, Team B Fouls = {teamb_foul}
            Team B FK-PK = {teamb_fk_pk}, Team A Fouls = {teama_foul}
            '''
            qc_excel_log(df[foul_mask | fk_mask], message, 45, 'fk_pk_foul_check')
            
            
            # logger_string_list = write_log(df[foul_mask | fk_mask])
            # logger_string = ''
            # for string in logger_string_list:
            #     logger_string = logger_string + string + '\n'
            # logger_obj.info(f'\n\n{logger_string}\n\n' + '*'*100)
    
        except:
            pass

In [16]:
fk_pk_foul_check()

Foul to fk-pk not equal
Team A FK-PK = 14, Team B Fouls = 15
Team B FK-PK = 16, Team A Fouls = 16


,team,jersey_number,action,notation,start_grid,end_grid,timestamp,foot,special_attribute,half
283,B,20,ST,0,68,X,0:09:11,R,F,FHN
285,A,4,C,0,14,51,0:09:42,R,FK,FHN
412,B,6,GD,0,28,X,0:15:21,R,YC,FHN
416,A,8,SP,0,53,31,0:16:16,R,FK,FHN
539,B,9,OFF,1,32,X,0:21:41,R,X,FHN
699,B,4,SL,0,64,X,0:28:02,R,F,FHN
701,A,4,LP,1,17,48,0:28:23,R,FK,FHN
951,B,14,ST,0,56,X,0:41:48,R,F,FHN
953,A,5,LP,1,25,38,0:42:14,L,FK,FHN
992,B,9,OFF,1,54,X,0:43:37,R,X,FHN


## Corner QC function

In [17]:
def corner_qc():
    """
    The purpose of this function
    1. If there are any corners which didn't start from the corner grids
    2. If there are any unassigned corners within the crosses
    
    """

    # To check if there were any passes which didn't start from the correct corner start grid(01,71) designated as corners
    false_cn = df[(df['special_attribute'] == 'CN') & ~(df['start_grid'].isin(['01', '1', '71']))]
    false_cn_count = false_cn['action'].count()
    if false_cn_count != 0:
        print("This Corner started from a false start grid. Check!")
        display(false_cn)
        message = 'This Corner started from a false start grid. Please check!'
        qc_excel_log(false_cn, message, 30, 'corner_grid_check')

        # logger_obj.info('\n\nThis Corner started from a false start grid. Check!\n' + f'{write_log(false_cn)}\n\n' + '*'*100)
    else:
        print("No corner false start grid error")
        # logger_obj.info("\n\nNo corner false start grid error\n\n" + '*'*100)

    

    print(f"\n{'*' * 90}\n")
    
    
    # To check if any crosses starting from (01, 71) are corners
    corner_in_cross = df[(df['special_attribute'] != 'CN') & (df['start_grid'].isin(['01', '1', '71'])) & (df['action'] == 'C')]
    corner_in_cross_count = corner_in_cross['action'].count()
    if corner_in_cross_count != 0:
        print(f"{corner_in_cross_count} cross found with corner grid. Check if they are corners")
        display(corner_in_cross)
        message = f'{corner_in_cross_count} cross found with corner grid. Check if they are corners'
        qc_excel_log(corner_in_cross, message, 30, 'cross_check')

        # logger_obj.info(f'\n\n{corner_in_cross_count} cross found with corner grid. Check if they are crosses\n'\
        #                 + f'{write_log(corner_in_cross)}\n\n' + '*'*100)
    else:
        print("No corners in crosses")
        # logger_obj.info("\n\nNo corners in crosses\n\n" + '*'*100)

In [18]:
corner_qc()

No corner false start grid error

******************************************************************************************

No corners in crosses


## Key passes and Assist QC

In [19]:
index_list = df[df['action'].isin(['CS', 'LS'])].index

In [20]:
# for index in index_list:
#     display(df.loc[index-5:index])

## Goal Kick QC

In [21]:
def gk_qc():
    '''
    This function checks if the GK, GH and GT were taken from inside their respective grids or not
    '''

    # Checking if GK is taken from outside goal area
    wrong_goalkick = df[(df['special_attribute'] == 'GK') & ~(df['start_grid'].isin(['40', '50']))]
    wrong_goalkick_count = wrong_goalkick['action'].count()
    if wrong_goalkick_count > 0:
        print('GK QC-1\nThis goal kick is taken outside the goal area. Please check!')
        display(wrong_goalkick)   
        message = 'GK QC-1\nThis goal kick is taken outside the goal area. Please check!'
        qc_excel_log(wrong_goalkick, message, 30, 'GK QC-1')


        # logger_obj.info('\n\nGK QC-1\nThis goal kick is taken outside the goal area. Please check!\n'\
        #                + f'{write_log(wrong_goalkick)}\n' + '*'*100)
    else:
        print('GK QC-1 done.')
        # logger_obj.info('\n\nGK QC-1 done.\n\n' + '*'*100)

    # Chceking GH or GT is taken from outside penalty area
    gk_d_grids = ['29','30','39','40','49','50','59','60']
    wrong_goalkeeper = df[(df['action'].isin(['GH', 'GT'])) & ~(df['start_grid'].isin(gk_d_grids))]
    wrong_goalkeeper_count = wrong_goalkeeper['action'].count()
    if wrong_goalkeeper_count > 0:
        print('GK QC-2\nThis GH or GT taken outside penalty area. Please check!')
        display(wrong_goalkeeper)
        message = 'GK QC-2\nThis GH or GT taken outside penalty area. Please check!'
        qc_excel_log(wrong_goalkeeper, message, 30, 'GK QC-2')

        # logger_obj.info('\n\nGK QC-2\nThis GH or GT taken outside penalty area. Please check!\n'\
        #                + f'{write_log(wrong_goalkeeper)}\n' + '*'*100)
        
    else:
        print('GK QC-2 done.')
        # logger_obj.info('\n\nGK QC-2 done.\n\n' + '*'*100)

In [22]:
gk_qc()

GK QC-1 done.
GK QC-2 done.


## Penalty QC

In [23]:
def penalty_qc():
    '''
    This function checks if the PK was taken from inside their respective grids or not
    '''

    # Checking if PK is taken from the appropriate grid location(32, 42)
    wrong_pk = df[(df['special_attribute'] == 'PK') & ~(df['start_grid'].isin(['32', '42']))]
    wrong_pk_count = wrong_pk['action'].count()
    if wrong_pk_count > 0:
        print('This penalty kick is taken outside the penalty area(32, 42). Please check!')
        display(wrong_pk)   
        message = 'This penalty kick is taken outside the penalty area(32, 42). Please check!'
        qc_excel_log(wrong_pk, message, 30, 'Penalty_check')

        # logger_obj.info('\n\nThis penalty kick is taken outside the penalty area(32, 42). Please check!\n'\
        #                + f'{write_log(wrong_pk)}\n' + '*'*100)
    else:
        print('PK QC done.')
        # logger_obj.info('\n\nPK QC done.\n\n' + '*'*100)

In [24]:
penalty_qc()

PK QC done.


## To-do

In [25]:
# penalty grid check - done
# correct grid location - done
# check for unsuccessful interception - done
# match summary - team, jersey number, type of pass
# Add HB condition in non defensive action - done
# corner previous action check
# match report - goalsocerer and assist, shot and respective key pass (timestamp-wise) - done
# check unsuccessful AD and GD add 1 automaically - done

# change to excel log - done
# Rather than displaying all fk-pk and foul, only show error ones
# add more options to game-time in renumeration function
# right now when fk-pk--foul count inequal for both teams, nothing gets logged to the excel file, work on this

## Miscellanous checks

In [26]:
def miscellanous_checks():
    '''
    This function checks if 
    1. There are any unsuccessful interceptions(IN-0).
    '''
    unsuccessful_interception = df[(df['action'].isin(['IN', 'XIN'])) & (df['notation'] == '0')]
    unsuccessful_interception_count = unsuccessful_interception['action'].count()

    if (unsuccessful_interception_count != 0):
        print('An unsuccessful interception was tagged. Please check!')
        display(unsuccessful_interception)
        message = 'An unsuccessful interception was tagged. Please check!'
        qc_excel_log(unsuccessful_interception, message, 30, 'unsuccessful_interception_check')

        # logger_obj.info(f"\n\nAn unsuccessful interception was tagged. Please check! \n{write_log(unsuccessful_interception)}\n\n" + '*'*100)

    else:
        print('No unsuccessful interceptions.')


In [27]:
miscellanous_checks()

No unsuccessful interceptions.


## Key pass-shot, assist-goal report

In [28]:
# assist_mask = ((df['team'] == team) & (df['action'].isin(['SP', 'LP', 'C', 'TB', 'XSP', 'XLP', 'XC', 'XTB'])) & (df['notation'] == '3'))
# goal_scorer_mask = ((df['team'] == team) & (df['action'].isin(['CS', 'LS'])) & (df['notation'].isin(['4'])))

# df[goal_scorer_mask | assist_mask]



# non_goal_shots_mask = ((df['team'] == team) & (df['action'].isin(['CS', 'LS'])) & ~(df['notation'].isin(['4'])))
# key_pass_mask = ((df['action'].isin(['SP', 'LP', 'C', 'TB', 'XSP', 'XLP', 'XC', 'XTB'])) & (df['notation'] == '2'))

# df[non_goal_shots_mask | key_pass_mask]

In [29]:
match_report_path_xlsx = "../match_report/Match_ID_" + str(match_idd) + "_Time_" + time.strftime("%H-%M-%S") + ".xlsx"

def key_assist_shot_detail(team, team_name):

    global shot_pass_df
    
    key_assist_pass_mask = ((df['team'] == team) & (df['action'].isin(['SP', 'LP', 'C', 'TB', 'XSP', 'XLP', 'XC', 'XTB'])) & (df['notation'].isin(['2', '3'])))
    shot_goal_mask = ((df['team'] == team) & (df['action'].isin(['CS', 'LS', 'H'])))
    shot_pass_df = df[key_assist_pass_mask | shot_goal_mask].copy()



    # cleaning and rearranging shot_pass_df
    
    # creating 'receiver' col
    index_list = []
    
    for index, row in shot_pass_df.iterrows():
        if row['action'] in ['XSP', 'XLP', 'XC', 'XTB']:
            shot_pass_df.at[index - 1, 'receiver'] = row['jersey_number']
            index_list.append(index)
    
    shot_pass_df.drop(index_list, inplace = True)
    
    
    # creating 'action_taken' col
    action_dict = {
        'TB' : {'2' : 'Key Through Pass', '3' : 'Through Pass Assist'},
        'SP' : {'2' : 'Key Short Pass', '3' : 'Short Pass Assist'},
        'LP' : {'2' : 'Key Long Pass', '3' : 'Long Pass Assist'},
        'C' : {'2' : 'Key Cross', '3' : 'Cross Assist'},
        'CS' : {'0' : 'Off-Target Close Shot',
               '1' : 'Simple Close Shot',
               '2' : 'CS-Hitting Cross bar/Post',
               '3' : 'Brilliant Close Shot',
               '4' : 'Close Shot Goal'},
        'LS' : {'0' : 'Off-Target Long Shot',
               '1' : 'Simple Long Shot',
               '2' : 'LS-Hitting Cross bar/Post',
               '3' : 'Brilliant Long Shot',
               '4' : 'Long Shot Goal'},
        'H' : {'0' : 'Off-Target Header',
               '1' : 'Simple Header',
               '2' : 'Header-Hitting Cross bar/Post',
               '3' : 'Brilliant Header',
               '4' : 'Header Goal'}
    }
    
    def action_to_words(row):
        return action_dict[row['action']][row['notation']]
        
    shot_pass_df['action_taken'] = shot_pass_df[['action', 'notation']].apply(action_to_words, axis = 1)
    
    # creating 'half' col
    half_dict = {
        'FHN' : 'First Half',
        'FHI' : 'First Half Injury Time',
        'SHN' : 'Second Half',
        'SHI' : 'Second Half Injury Time',
        'ET1N' : 'Extra-Time First Half',
        'ET1I' : 'Extra-Time First Half Injury Time',
        'ET2N' : 'Extra-Time Second Half',
        'ET2I' : 'Extra-Time Second Half Injury Time',
        'PK' : 'Penalty Shoot-Out'
    }
    shot_pass_df['half'] = shot_pass_df['half'].map(half_dict)
    
    
    # creating special_attribute col
    sp_att_dict = {
        'X' : np.nan,
        'FK' : 'Free Kick',
        'PK' : 'Penalty Kick',
        'GK' : 'Goal Kick',
        'CN' : 'Corner Kick'
    } 
    shot_pass_df['special_attribute'] = shot_pass_df['special_attribute'].map(sp_att_dict)

    # dropping and reordering columns
    shot_pass_df.drop(columns = ['action', 'notation', 'start_grid', 'end_grid', 'foot', 'team'], inplace = True)
    
    desired_order = ['timestamp', 'jersey_number', 'special_attribute', 'action_taken', 'receiver', 'half']
    
    shot_pass_df = shot_pass_df[desired_order]
    
    # writing the match report to an excel file
    path = r'..\match_report'
    if not os.path.exists(path):
        os.makedirs(path)
        
    sheet_name = team_name
    
    if os.path.exists(match_report_path_xlsx):
        wb = load_workbook(match_report_path_xlsx)
        ws = wb.create_sheet(sheet_name)                
        
    else:
        wb = Workbook()
        ws = wb.active
        ws.title = f'{sheet_name}'

    for r in dataframe_to_rows(shot_pass_df, index=False, header=True):
        ws.append(r)
        
    for cell in ws[1]:
        cell.style = 'Pandas'

    for col in ['A', 'B', 'C', 'D', 'E', 'F']:
        ws.column_dimensions[col].width = 25

    assist_goal_list = ['Through Pass Assist', 'Short Pass Assist', 'Long Pass Assist', 'Cross Assist',
                         'Close Shot Goal', 'Long Shot Goal', 'Header Goal']
    for row_idx in range(2, shot_pass_df.shape[0] + 2):
        action_t_cell = ws.cell(row = row_idx, column = 4)
        if action_t_cell.value in assist_goal_list:
            action_t_cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
    
    wb.save(match_report_path_xlsx)



In [30]:
key_assist_shot_detail('A', team_a_name)
key_assist_shot_detail('B', team_b_name)

## GD-AD correction

In [31]:
gd_ad_idx_list = []
def gd_ad_idx_list_func(df):
    if '1' not in df['notation'].unique():
        for i in df.iloc[[-2,-1]].index:
            gd_ad_idx_list.append(i)
            
for filter in [['GD', 'XGD'], ['AD', 'XAD']]:
    tmsp_list = df[(df['action'].isin(filter))]['timestamp'].unique()
    
    for tmsp in tmsp_list:
        gd_df = df[(df['action'].isin(filter)) & (df['timestamp'] == tmsp)]
        row_count = gd_df['action'].count()
        if row_count > 4:
            divs = int(row_count/4)
            i = 1
            while i <= divs:
                gd_dff = gd_df.iloc[4*(i-1) : 4*i]
                i += 1
                gd_ad_idx_list_func(gd_dff)
        else:
            gd_ad_idx_list_func(gd_df)
            
                        
    for idx in gd_ad_idx_list:
        df.at[idx, 'notation'] = '1'

write(write_path, df)

wstring file successfully created.
